In [9]:
#保存モデルを分かりやすく，valを含めたK-FoldCV法導入
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score,
    recall_score, roc_auc_score, roc_curve, confusion_matrix
)
import matplotlib.pyplot as plt
import seaborn as sns
from fttransformer import FTTransformer
import os


# --------------------------------------------
# 設定
# --------------------------------------------
DATA_PATH = "/home/osajima/ドキュメント/program/Transformer_Study/fttransformer_pytorch/cardio_train.csv"
TEST_RATIO = 0.2       # train+val : test = 0.8 : 0.2
K_FOLDS = 5            # K-Fold CV の分割数
EPOCHS = 5
BATCH_SIZE = 128
LR = 3e-4
SAVE_DIR = os.path.join(os.getcwd(), "save_model")
os.makedirs(SAVE_DIR, exist_ok=True)

# --------------------------------------------
# データ読み込みと前処理
# --------------------------------------------
data = pd.read_csv(DATA_PATH, sep=';')
data = data.drop(columns=["id"])
X = data.drop(columns=["cardio"])
y = data["cardio"]

categorical_cols = ["gender", "cholesterol", "gluc", "smoke", "alco", "active"]
continuous_cols = ["age", "height", "weight", "ap_hi", "ap_lo"]

# Label Encoding for categorical
for col in categorical_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
# Standardize continuous
scaler = StandardScaler()
X[continuous_cols] = scaler.fit_transform(X[continuous_cols])

# Hold out test set
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=TEST_RATIO, random_state=42, stratify=y
)

def get_data_loader(X, y, shuffle=False):
    class CardioDataset(Dataset):
        def __init__(self, X, y):
            self.X_categ = torch.tensor(X[categorical_cols].values, dtype=torch.long)
            self.X_cont = torch.tensor(X[continuous_cols].values, dtype=torch.float32)
            self.y = torch.tensor(y.values, dtype=torch.float32).unsqueeze(1)

        def __len__(self):
            return len(self.y)

        def __getitem__(self, idx):
            return self.X_categ[idx], self.X_cont[idx], self.y[idx]

    return DataLoader(CardioDataset(X, y), batch_size=BATCH_SIZE, shuffle=shuffle)

# テスト用データローダ
test_loader = get_data_loader(X_test, y_test, shuffle=False)

# 可視化関数
# 画像を保存したいディレクトリを指定（なければ自動作成）
IMAGE_DIR = "/home/osajima/ドキュメント/program/Transformer_Study/fttransformer_pytorch/save_confusion_matrix_images"
os.makedirs(IMAGE_DIR, exist_ok=True)

def plot_all_metrics(train_losses, train_accs, val_accs,
                    y_true, y_pred, y_prob,
                    fold, epoch, val_acc, test_acc,
                    save_dir):
    img_name = (
        f"confusion_matrix_epoch{best_epoch}"
        f"_fold{fold}"
        f"_valacc{best_val_acc:.3f}"
        f"_testacc{test_acc:.3f}.png"
    )
    img_path = os.path.join(IMAGE_DIR, img_name)
    cm = confusion_matrix(y_true, y_pred)
    fpr, tpr, _ = roc_curve(y_true, y_prob)
    auc = roc_auc_score(y_true, y_prob)
    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)

    fig, axs = plt.subplots(2, 2, figsize=(12, 10))
    # ① Training Loss
    axs[0, 0].plot(range(1, len(train_losses) + 1), train_losses, marker='o')
    axs[0, 0].set_title("Training Loss")
    axs[0, 0].set_xlabel("Epoch")
    axs[0, 0].set_ylabel("Loss")
    axs[0, 0].grid(True)

    # ② Accuracy
    axs[0, 1].plot(range(1, len(train_accs) + 1), train_accs, marker='o', label='Train Acc')
    axs[0, 1].plot(range(1, len(val_accs) + 1), val_accs, marker='x', label='Val Acc')
    axs[0, 1].set_title("Accuracy")
    axs[0, 1].set_xlabel("Epoch")
    axs[0, 1].set_ylabel("Accuracy")
    axs[0, 1].legend()
    axs[0, 1].grid(True)

    # ③ Confusion Matrix
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axs[1, 0])
    axs[1, 0].set_title("Confusion Matrix")
    axs[1, 0].set_xlabel("Predicted")
    axs[1, 0].set_ylabel("True")
    axs[1, 0].set_xticklabels(['No Cardio', 'Cardio'])
    axs[1, 0].set_yticklabels(['No Cardio', 'Cardio'], rotation=0)

    # ④ ROC Curve
    axs[1, 1].plot(fpr, tpr, label=f'AUC = {auc:.3f}')
    axs[1, 1].plot([0, 1], [0, 1], 'k--')
    axs[1, 1].set_title('ROC Curve')
    axs[1, 1].set_xlabel('FPR')
    axs[1, 1].set_ylabel('TPR')
    axs[1, 1].legend(loc='lower right')
    axs[1, 1].text(0.05, 0.2,
                f"Acc: {acc:.3f}\nF1: {f1:.3f}\nPrec: {precision:.3f}\nRec: {recall:.3f}",
                fontsize=10, bbox=dict(facecolor='white', alpha=0.7))

    plt.tight_layout()
    plt.show()
    img_name = (
        f"confusion_matrix_epoch{epoch}"
        f"_fold{fold}"
        f"_valacc{val_acc:.3f}"
        f"_testacc{test_acc:.3f}.png"
    )
    img_path = os.path.join(save_dir, img_name)
    plt.savefig(img_path, dpi=200, bbox_inches="tight")
    plt.close()

# K-Fold CV + テスト評価
kf = KFold(n_splits=K_FOLDS, shuffle=True, random_state=42)
fold_results = []

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for fold, (tr_idx, va_idx) in enumerate(kf.split(X_train_val), 1):
    print(f"\n=== Fold {fold}/{K_FOLDS} ===")
    X_tr, y_tr = X_train_val.iloc[tr_idx], y_train_val.iloc[tr_idx]
    X_va, y_va = X_train_val.iloc[va_idx], y_train_val.iloc[va_idx]
    train_loader = get_data_loader(X_tr, y_tr, shuffle=True)
    val_loader   = get_data_loader(X_va, y_va, shuffle=False)

    # モデル初期化
    model = FTTransformer(
        categories=[X[col].nunique() for col in categorical_cols],
        num_continuous=len(continuous_cols),
        dim=64, depth=6, heads=8,
        ff_dropout=0.2, attn_dropout=0.2
    ).to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.AdamW(model.parameters(), lr=LR)

    best_val_acc = 0.0
    best_epoch   = 0 
    train_losses, train_accs, val_accs = [], [], []

    # 学習ループ
    for epoch in range(1, EPOCHS + 1):
        model.train()
        running_loss = 0.0
        for x_cat, x_cont, labels in train_loader:
            x_cat, x_cont, labels = x_cat.to(device), x_cont.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(x_cat, x_cont)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        avg_loss = running_loss / len(train_loader)

        # train accuracy
        model.eval()
        train_correct, train_total = 0, 0
        with torch.no_grad():
            for x_cat, x_cont, labels in train_loader:
                preds = (torch.sigmoid(model(x_cat.to(device), x_cont.to(device))) > 0.5).float()
                train_correct += (preds.cpu() == labels).sum().item()
                train_total += labels.size(0)
        train_acc = train_correct / train_total

        # validation
        all_preds, all_labels, all_probs = [], [], []
        with torch.no_grad():
            for x_cat, x_cont, labels in val_loader:
                probs = torch.sigmoid(model(x_cat.to(device), x_cont.to(device))).cpu().numpy()
                preds = (probs > 0.5).astype(int)
                all_probs.extend(probs.flatten())
                all_preds.extend(preds.flatten())
                all_labels.extend(labels.numpy())
        val_acc = accuracy_score(all_labels, all_preds)

        train_losses.append(avg_loss)
        train_accs.append(train_acc)
        val_accs.append(val_acc)

        print(f"[Fold {fold}][Epoch {epoch}/{EPOCHS}] Loss: {avg_loss:.4f} | Val Acc: {val_acc:.4f}")

        # ベストモデル保存（epoch→fold→valacc の順）
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_epoch   = epoch
            # 必要なら SAVE_DIR の定義はループの外に
            filename = (
                f"best_model_epoch{best_epoch}"
                f"_fold{fold}"
                f"_valacc{best_val_acc:.3f}.pth"
            )
            save_path = os.path.join(SAVE_DIR, filename)
            torch.save(model.state_dict(), save_path)
            print(f"  >>> Saved {filename}")


    print(f"Fold {fold} Best Val Acc: {best_val_acc:.4f}")

    # テスト評価
    best_model = FTTransformer(
        categories=[X[col].nunique() for col in categorical_cols],
        num_continuous=len(continuous_cols),
        dim=64, depth=6, heads=8,
        ff_dropout=0.2, attn_dropout=0.2
    ).to(device)
    best_model.load_state_dict(torch.load(os.path.join(os.getcwd(), filename)))
    best_model.eval()

    y_true, y_pred, y_prob = [], [], []
    with torch.no_grad():
        for x_cat, x_cont, labels in test_loader:
            probs = torch.sigmoid(best_model(x_cat.to(device), x_cont.to(device))).cpu().numpy()
            preds = (probs > 0.5).astype(int)
            y_prob.extend(probs.flatten())
            y_pred.extend(preds.flatten())
            y_true.extend(labels.numpy())

    # テストメトリクス
    test_acc = accuracy_score(y_true, y_pred)
    test_f1 = f1_score(y_true, y_pred)
    test_precision = precision_score(y_true, y_pred)
    test_recall = recall_score(y_true, y_pred)
    test_auc = roc_auc_score(y_true, y_prob)

    print(f"--- Test Evaluation Fold {fold} ---")
    print(f"Accuracy : {test_acc:.4f}, F1: {test_f1:.4f}, Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, AUC: {test_auc:.4f}")
    # ── リネームではなく、直接保存 ──
    save_test_path = os.path.join(
        SAVE_DIR,
        f"best_model_fold{fold}_epoch{best_epoch}_valacc{best_val_acc:.3f}_testacc{test_acc:.3f}.pth"
    )
    torch.save(best_model.state_dict(), save_test_path)
    print(f"  >>> Model saved with testacc: {os.path.basename(save_test_path)}")
    # Foldごとのプロット（情報を渡す）
    plot_all_metrics(
        train_losses, train_accs, val_accs,
        y_true, y_pred, y_prob,
        fold, best_epoch, best_val_acc, test_acc,
        IMAGE_DIR
    )

    fold_results.append((test_acc, test_f1, test_auc, test_recall))

# まとめ表示
print("\n=== All Folds Test Results ===")
avg_acc = np.mean([r[0] for r in fold_results])
avg_f1  = np.mean([r[1] for r in fold_results])
avg_auc = np.mean([r[2] for r in fold_results])
avg_rec = np.mean([r[3] for r in fold_results])
print(f"Average Accuracy: {avg_acc:.4f}, Average F1: {avg_f1:.4f}, Average AUC: {avg_auc:.4f}, Average Recall: {avg_rec:.4f}")



=== Fold 1/5 ===
[Fold 1][Epoch 1/5] Loss: 0.5622 | Val Acc: 0.7316
  >>> Saved best_model_epoch1_fold1_valacc0.732.pth
[Fold 1][Epoch 2/5] Loss: 0.5472 | Val Acc: 0.7314
[Fold 1][Epoch 3/5] Loss: 0.5449 | Val Acc: 0.7276
[Fold 1][Epoch 4/5] Loss: 0.5437 | Val Acc: 0.7294
[Fold 1][Epoch 5/5] Loss: 0.5430 | Val Acc: 0.7312
Fold 1 Best Val Acc: 0.7316


FileNotFoundError: [Errno 2] No such file or directory: '/home/osajima/ドキュメント/program/Transformer_Study/fttransformer_pytorch/best_model_epoch1_fold1_valacc0.732.pth'